# Скачиваем данные и устанавливаем программы

In [ ]:
!wget https://raw.githubusercontent.com/AnyaAkhmatova/hse22_hw2/main/data/gms2.lst

--2022-10-19 13:31:30--  https://raw.githubusercontent.com/AnyaAkhmatova/hse22_hw2/main/data/gms2.lst
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 220305 (215K) [text/plain]
Saving to: ‘gms2.lst’

gms2.lst            100%[===================>] 215.14K  --.-KB/s    in 0.003s  

2022-10-19 13:31:30 (67.6 MB/s) - ‘gms2.lst’ saved [220305/220305]



In [ ]:
!wget https://raw.githubusercontent.com/AnyaAkhmatova/hse22_hw1/main/data/scaffolds.fasta

--2022-10-19 13:31:32--  https://raw.githubusercontent.com/AnyaAkhmatova/hse22_hw1/main/data/scaffolds.fasta
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3971948 (3.8M) [text/plain]
Saving to: ‘scaffolds.fasta’

scaffolds.fasta     100%[===================>]   3.79M  --.-KB/s    in 0.01s   

2022-10-19 13:31:32 (264 MB/s) - ‘scaffolds.fasta’ saved [3971948/3971948]



In [ ]:
!wget https://raw.githubusercontent.com/AnyaAkhmatova/hse22_hw2/main/data/proteins.fasta

--2022-10-19 13:31:34--  https://raw.githubusercontent.com/AnyaAkhmatova/hse22_hw2/main/data/proteins.fasta
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1403607 (1.3M) [text/plain]
Saving to: ‘proteins.fasta’

proteins.fasta      100%[===================>]   1.34M  --.-KB/s    in 0.009s  

2022-10-19 13:31:34 (155 MB/s) - ‘proteins.fasta’ saved [1403607/1403607]



In [ ]:
!wget https://raw.githubusercontent.com/AnyaAkhmatova/hse22_hw2/main/data/scaffolds.hits_from_MIL_1.txt

--2022-10-19 13:31:39--  https://raw.githubusercontent.com/AnyaAkhmatova/hse22_hw2/main/data/scaffolds.hits_from_MIL_1.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 870940 (851K) [text/plain]
Saving to: ‘scaffolds.hits_from_MIL_1.txt’

scaffolds.hits_from 100%[===================>] 850.53K  --.-KB/s    in 0.007s  

2022-10-19 13:31:39 (117 MB/s) - ‘scaffolds.hits_from_MIL_1.txt’ saved [870940/870940]



In [ ]:
!wget https://raw.githubusercontent.com/AnyaAkhmatova/hse22_hw2/main/data/scaffolds.hits_from_SwissProt.txt

--2022-10-19 13:31:42--  https://raw.githubusercontent.com/AnyaAkhmatova/hse22_hw2/main/data/scaffolds.hits_from_SwissProt.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 66190 (65K) [text/plain]
Saving to: ‘scaffolds.hits_from_SwissProt.txt’

scaffolds.hits_from 100%[===================>]  64.64K  --.-KB/s    in 0.001s  

2022-10-19 13:31:42 (49.6 MB/s) - ‘scaffolds.hits_from_SwissProt.txt’ saved [66190/66190]



## Скачиваем геном близкородственной бактерии T.oleivorans

Геном, последовательности генов (нт) и белков (протеом) для бактерии Thalassolituus oleivorans MIL-1
https://www.ncbi.nlm.nih.gov/nuccore/HF680312

In [ ]:
!sh -c "$(curl -fsSL ftp://ftp.ncbi.nlm.nih.gov/entrez/entrezdirect/install-edirect.sh)"


Entrez Direct has been successfully downloaded and installed.


To activate EDirect for this terminal session, please execute the following:

export PATH=${PATH}:${HOME}/edirect



In [ ]:
!$HOME/edirect/efetch -db nuccore -id HF680312 -format gb  >  T_oleivorans_MIL_1.gbk

In [ ]:
!pip install biopython

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Аннотация генома

In [135]:
from Bio import SeqIO

scaffolds = dict() 

for record in SeqIO.parse("/content/scaffolds.fasta", "fasta"):
  record.annotations['molecule_type'] = 'DNA'
  record.annotations['date'] = "19-OCT-2022"
  scaffolds[record.id] = record

In [136]:
print(scaffolds["scaffold1_cov231"])

ID: scaffold1_cov231
Name: scaffold1_cov231
Description: scaffold1_cov231
Number of features: 0
/molecule_type=DNA
/date=19-OCT-2022
Seq('CACTTTTTTAGTAGGTCGACCTCTTGCTTAAGCTTGCGATTTTCGGCTTCTAAT...TCA')


## Добавляем координаты генов и трансляции генов

In [137]:
from Bio.SeqFeature import SeqFeature, FeatureLocation

for record in SeqIO.parse("/content/proteins.fasta", "fasta"):
  gene_description = record.description.split() # id, scaffold, start, end, strand, ...
  id = gene_description[0]
  scaffold = gene_description[1]
  start = int(gene_description[2])
  end = int(gene_description[3])
  strand = 1 if gene_description[4] == '+' else -1
  
  feature = SeqFeature(FeatureLocation(start, end, strand=strand), type="CDS")
  feature.qualifiers['locus_tag'] = [id]
  feature.qualifiers['translation'] = [record.seq]
  scaffolds[scaffold].features.append(feature)

In [138]:
print(scaffolds["scaffold1_cov231"])

ID: scaffold1_cov231
Name: scaffold1_cov231
Description: scaffold1_cov231
Number of features: 3536
/molecule_type=DNA
/date=19-OCT-2022
Seq('CACTTTTTTAGTAGGTCGACCTCTTGCTTAAGCTTGCGATTTTCGGCTTCTAAT...TCA')


In [139]:
print(scaffolds["scaffold1_cov231"].features[0])

type: CDS
location: [3:299](-)
qualifiers:
    Key: locus_tag, Value: ['1']
    Key: translation, Value: [Seq('MPKYTKPRKTWQYSNEFKVKAVQLSLIEGIQVKEVAVTLDIHPFMLSRWRKEYR...LKK')]



## Добавляем функции белков (из бактерии MIL-1)

In [140]:
import pandas as pd
import numpy as np

df = pd.read_csv("/content/scaffolds.hits_from_MIL_1.txt", sep='\t', header=None, names=
                 ["Query sequence ID", "Subject sequence ID", "Percentage of identical positions", 
                  "Alignment length", "Number of mismatches", "Number of gap openings",
                  "Start position of alignment on query sequence", "End position of alignment on query sequence",
                  "Start position of alignment on subject sequence", "End position of alignment on subject sequence",
                  "Expect value", "Bit score"])
df = df[pd.DataFrame([line for line in df['Subject sequence ID'].str.split('_').values]).loc[:, 2].str[0:3] == 'CCU']
df = df.sort_values(by=["Bit score"], ascending=False).drop_duplicates(subset=["Query sequence ID"]).sort_values(by=["Query sequence ID"])
df

,Query sequence ID,Subject sequence ID,Percentage of identical positions,Alignment length,Number of mismatches,Number of gap openings,Start position of alignment on query sequence,End position of alignment on query sequence,Start position of alignment on subject sequence,End position of alignment on subject sequence,Expect value,Bit score
0,1,lcl|HF680312.1_prot_CCU71653.1_1204,98.990,99,1,0,1,99,1,99,8.080000e-68,196.0
7,3,lcl|HF680312.1_prot_CCU71928.1_1479,100.000,173,0,0,1,173,1,173,1.660000e-130,359.0
8,5,lcl|HF680312.1_prot_CCU71924.1_1475,97.143,35,1,0,1,35,1,35,1.510000e-19,75.1
9,6,lcl|HF680312.1_prot_CCU71587.1_1138,98.024,253,5,0,1,253,29,281,1.320000e-176,484.0
11,7,lcl|HF680312.1_prot_CCU71586.1_1137,100.000,391,0,0,1,391,1,391,0.000000e+00,810.0
...,...,...,...,...,...,...,...,...,...,...,...,...
10491,3606,lcl|HF680312.1_prot_CCU73035.1_2586,100.000,35,0,0,1,35,38,72,9.440000e-22,76.6
10496,3607,lcl|HF680312.1_prot_CCU70835.1_386,45.033,151,78,4,2,150,2735,2882,1.070000e-23,92.8
10505,3608,lcl|HF680312.1_prot_CCU71715.1_1266,100.000,63,0,0,1,63,160,222,6.030000e-39,130.0
10519,3611,lcl|HF680312.1_prot_CCU73034.1_2585,100.000,48,0,0,1,48,428,475,2.570000e-29,103.0


In [142]:
# создадим словарь соответствия Query sequence ID -> Subject sequence ID
qs_id_ss_id = dict()
for _, record in df.iterrows():
  if record['Subject sequence ID'].split('_')[2][0:3] == 'CCU':
    qs_id_ss_id[record['Query sequence ID']] = record['Subject sequence ID'].split('_')[2]

print(qs_id_ss_id)

{1: 'CCU71653.1', 3: 'CCU71928.1', 5: 'CCU71924.1', 6: 'CCU71587.1', 7: 'CCU71586.1', 8: 'CCU71585.1', 9: 'CCU71584.1', 10: 'CCU71583.1', 11: 'CCU71582.1', 12: 'CCU71581.1', 13: 'CCU71580.1', 14: 'CCU71579.1', 15: 'CCU71578.1', 16: 'CCU71577.1', 17: 'CCU71576.1', 18: 'CCU71575.1', 19: 'CCU71574.1', 20: 'CCU71573.1', 21: 'CCU71572.1', 22: 'CCU71571.1', 23: 'CCU71570.1', 24: 'CCU71569.1', 25: 'CCU71568.1', 26: 'CCU71567.1', 27: 'CCU71566.1', 28: 'CCU71564.1', 29: 'CCU71562.1', 30: 'CCU71561.1', 31: 'CCU71560.1', 32: 'CCU71559.1', 33: 'CCU71558.1', 34: 'CCU71557.1', 35: 'CCU71556.1', 36: 'CCU71555.1', 37: 'CCU71554.1', 38: 'CCU71553.1', 39: 'CCU71552.1', 40: 'CCU71551.1', 41: 'CCU71550.1', 42: 'CCU71549.1', 43: 'CCU71548.1', 45: 'CCU71547.1', 46: 'CCU71546.1', 47: 'CCU71545.1', 48: 'CCU71544.1', 49: 'CCU71543.1', 50: 'CCU71542.1', 51: 'CCU71541.1', 52: 'CCU71540.1', 53: 'CCU71539.1', 54: 'CCU71538.1', 55: 'CCU71537.1', 56: 'CCU71536.1', 57: 'CCU71535.1', 58: 'CCU71534.1', 59: 'CCU71533.1'

In [143]:
mil_1_genome = SeqIO.read("/content/T_oleivorans_MIL_1.gbk", "genbank")

In [144]:
# создадим словарь соответствия Subject sequence ID -> product(функция белка)
ss_id_product = dict()
for mil_1_feature in mil_1_genome.features:
  if ('protein_id' not in mil_1_feature.qualifiers) or ('product' not in mil_1_feature.qualifiers):
    continue
  ss_id_product[mil_1_feature.qualifiers['protein_id'][0]] = mil_1_feature.qualifiers['product']
  
print(ss_id_product)

{'CCU70450.1': ['chromosomal replication initiator protein'], 'CCU70451.1': ['DNA polymerase III subunit beta'], 'CCU70452.1': ['DNA replication and repair protein RecF'], 'CCU70453.1': ['DNA gyrase subunit B'], 'CCU70454.1': ['hypothetical protein'], 'CCU70455.1': ['hypothetical protein'], 'CCU70456.1': ['hypothetical protein'], 'CCU70457.1': ['hypothetical protein'], 'CCU70458.1': ['hypothetical protein'], 'CCU70459.1': ['hypothetical protein'], 'CCU70460.1': ['transposase orfA'], 'CCU70461.1': ['transposase orfB'], 'CCU70462.1': ['hypothetical protein'], 'CCU70463.1': ['metal-dependent hydrolase'], 'CCU70464.1': ['small-conductance mechanosensitive channel-like protein'], 'CCU70465.1': ['hypothetical protein'], 'CCU70466.1': ['acyltransferase'], 'CCU70467.1': ['histidinol phosphatase'], 'CCU70468.1': ['glycyl-tRNA synthetase, beta subunit'], 'CCU70469.1': ['glycyl-tRNA synthetase, alpha subunit'], 'CCU70470.1': ['spermidine synthase'], 'CCU70471.1': ['hypothetical protein'], 'CCU704

In [145]:
# будем итерироваться по скаффолдам, для каждого скаффолда - по найденным генам 
# если для гена найден аналог в геноме похожей бактерии, добавляем соответствующее название
# если для гена найден аналог в геноме похожей бактерии и у него указана функция белка, добавляем соответствующую функцию белка
count_added_notes = 0
count_added_products = 0
for scaffold_name, scaffold in scaffolds.items():
  for feature in scaffold.features:
    qs_id = int(feature.qualifiers['locus_tag'][0])
    if qs_id in qs_id_ss_id:
      feature.qualifiers['note'] = [qs_id_ss_id[qs_id]]
      count_added_notes += 1
      ss_id = qs_id_ss_id[qs_id]
      if ss_id in ss_id_product:
        feature.qualifiers['product'] = ss_id_product[ss_id]
        count_added_products += 1
print("Добавлено названий", count_added_notes)
print("Добавлено функций", count_added_products)

Добавлено названий 3332
Добавлено функций 3332


In [146]:
print(scaffolds["scaffold1_cov231"].features[0])

type: CDS
location: [3:299](-)
qualifiers:
    Key: locus_tag, Value: ['1']
    Key: note, Value: ['CCU71653.1']
    Key: product, Value: ['transposase orfA']
    Key: translation, Value: [Seq('MPKYTKPRKTWQYSNEFKVKAVQLSLIEGIQVKEVAVTLDIHPFMLSRWRKEYR...LKK')]



## Добавляем функции белков (из БД SwissProt)

In [147]:
df = pd.read_csv("/content/scaffolds.hits_from_SwissProt.txt", sep='\t', header=None, names=
                 ["Query sequence ID", "Subject sequence ID", "Percentage of identical positions", 
                  "Alignment length", "Number of mismatches", "Number of gap openings",
                  "Start position of alignment on query sequence", "End position of alignment on query sequence",
                  "Start position of alignment on subject sequence", "End position of alignment on subject sequence",
                  "Expect value", "Bit score"])
df = df.sort_values(by=["Bit score"], ascending=False).drop_duplicates(subset=["Query sequence ID"]).sort_values(by=["Query sequence ID"])
df

,Query sequence ID,Subject sequence ID,Percentage of identical positions,Alignment length,Number of mismatches,Number of gap openings,Start position of alignment on query sequence,End position of alignment on query sequence,Start position of alignment on subject sequence,End position of alignment on subject sequence,Expect value,Bit score
0,169,sp|P55546|Y4LF_SINFN,42.553,141,80,1,1,141,1,140,1.180000e-36,144.0
1,428,sp|Q06916|GUFA_MYXXA,43.802,242,136,0,71,312,13,254,2.330000e-47,162.0
69,591,sp|P08956|T1RK_ECOLI,40.701,1199,642,23,8,1171,5,1169,0.000000e+00,837.0
74,597,sp|Q9V1E9|KTHY_PYRAB,39.073,151,76,5,3,138,4,153,3.120000e-15,73.9
78,600,sp|P06990|T1SB_ECOLX,35.794,447,271,5,4,436,19,463,4.330000e-71,238.0
85,602,sp|A9VQW6|PTUB_BACMK,32.609,184,97,9,48,227,45,205,7.140000e-12,67.8
86,967,sp|P39213|INSN_SHIDY,46.392,97,51,1,7,103,17,112,3.770000e-27,99.0
89,968,sp|P39315|QOR2_ECOLI,55.705,149,66,0,3,151,132,280,3.510000e-48,160.0
90,974,sp|P37626|YHII_ECOLI,55.460,348,144,1,23,359,8,355,4.060000e-121,356.0
148,976,sp|P0AGH2|YHHJ_SHIFL,65.684,373,128,0,4,376,2,374,1.480000e-170,483.0


In [148]:
# создадим словарь соответствия Query sequence ID -> Subject sequence ID
qs_id_ss_id = dict()
for _, record in df.iterrows():
  qs_id_ss_id[record['Query sequence ID']] = record['Subject sequence ID'].split('|')[-1]

print(qs_id_ss_id)

{169: 'Y4LF_SINFN', 428: 'GUFA_MYXXA', 591: 'T1RK_ECOLI', 597: 'KTHY_PYRAB', 600: 'T1SB_ECOLX', 602: 'PTUB_BACMK', 967: 'INSN_SHIDY', 968: 'QOR2_ECOLI', 974: 'YHII_ECOLI', 976: 'YHHJ_SHIFL', 979: 'YACK_RHIML', 983: 'GLTS_HAEIN', 984: 'HERA_METJA', 986: 'T1S_SALTY', 989: 'T1RA_ECOLX', 992: 'INTQ_ECOLI', 998: 'VIP8_PSYL1', 999: 'ANR44_HUMAN', 1002: 'HIPA_ECOLI', 1003: 'HIPB_ECOLI', 1008: 'OLHYD_ELIME', 1011: 'HSP31_CANAL', 1013: 'CURA_ECOLI', 1203: 'ATPZ_PSEPK', 1216: 'Y459_NEIMB', 1372: 'Y2732_PARD8', 1376: 'INTF_ECOLI', 1409: 'T1S2_METJA', 1412: 'T1RK_ECOLI', 1491: 'BSHA_BACAN', 1661: 'UVR8_ARATH', 1688: 'PHZM_LYSAN', 1691: 'GALE_METJA', 1694: 'MCRB_ECOLI', 1701: 'DIND_ECOLI', 2011: 'Y1375_HAEIN', 2158: 'T1SE_ECOLX', 2160: 'T1RA_ECOLX', 2195: 'Y1178_METJA', 2498: 'T1SH_HAEIN', 2499: 'Y007_BORBU', 2555: 'HPA3_YEAST', 2974: 'NADR_ECOLI', 3257: 'PARE1_MYCTU', 3258: 'PARD_VIBCH', 3343: 'Y1057_VIBCH', 3365: 'HIGB2_MYCTU', 3371: 'MTD1_DESNO', 3437: 'Y2891_YERE8', 3439: 'YBHS_ECOLI', 3473: 'Y

Получить функции белков SwissProt из файла: https://ftp.uniprot.org/pub/databases/uniprot/current_release/knowledgebase/complete/uniprot_sprot.dat.gz

In [ ]:
!wget https://ftp.uniprot.org/pub/databases/uniprot/current_release/knowledgebase/complete/uniprot_sprot.dat.gz

--2022-10-19 13:54:46--  https://ftp.uniprot.org/pub/databases/uniprot/current_release/knowledgebase/complete/uniprot_sprot.dat.gz
Resolving ftp.uniprot.org (ftp.uniprot.org)... 128.175.240.195
Connecting to ftp.uniprot.org (ftp.uniprot.org)|128.175.240.195|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 642093634 (612M) [application/x-gzip]
Saving to: ‘uniprot_sprot.dat.gz’

uniprot_sprot.dat.g 100%[===================>] 612.35M  79.9MB/s    in 5.3s    

2022-10-19 13:54:51 (116 MB/s) - ‘uniprot_sprot.dat.gz’ saved [642093634/642093634]



In [ ]:
!gzip -d uniprot_sprot.dat.gz

In [ ]:
!grep -E '^(ID|DE   RecName: Full=)' uniprot_sprot.dat > uniprot_lines.txt

In [149]:
# создадим словарь соответствия Subject sequence ID -> product(функция белка)
ss_id_product = dict()

id = ''

with open('uniprot_lines.txt') as file:
  for line in file:
    tokens = line.split()
    if tokens[0] == 'ID':
      id = tokens[1]
    else:
      ss_id_product[id] = line.split('=')[-1][:-2]

ind = 0
count = 5
for ss_id, product in ss_id_product.items():
  if ind < count:
    print(ss_id, product)
    ind += 1

001R_FRG3G Putative transcription factor 001R
002L_FRG3G Uncharacterized protein 002L
002R_IIV3 Uncharacterized protein 002R
003L_IIV3 Uncharacterized protein 003L
003R_FRG3G Uncharacterized protein 3R


In [150]:
# будем итерироваться по скаффолдам, для каждого скаффолда - по найденным генам 
# если для гена найден аналог в геноме похожей бактерии, добавляем соответствующее название
# если для гена найден аналог в геноме похожей бактерии и у него указана функция белка, добавляем соответствующую функцию белка
count_added_notes = 0
count_added_products = 0
for scaffold_name, scaffold in scaffolds.items():
  for feature in scaffold.features:
    qs_id = int(feature.qualifiers['locus_tag'][0])
    if qs_id in qs_id_ss_id:
      feature.qualifiers['note'] = [qs_id_ss_id[qs_id]]
      count_added_notes += 1
      ss_id = qs_id_ss_id[qs_id]
      if ss_id in ss_id_product:
        feature.qualifiers['product'] = ss_id_product[ss_id]
        count_added_products += 1
print("Добавлено названий", count_added_notes)
print("Добавлено функций", count_added_products)

Добавлено названий 53
Добавлено функций 53


In [151]:
# записываем данные в файл
SeqIO.write(scaffolds.values(), "GENOME.gbk", "genbank")

65